<a href="https://colab.research.google.com/github/Squigspear/xCyberLLM/blob/main/Tutorial_2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial showcases **Retrieval Augmentation Generation** with a LLM, where a document (CCOP) is referred to when the model is queried.

It can be ran using either:

  >Open-sourced LLAMA 2 7B 5 bit quantized model *or*

  >Microsoft Azure OpenAI.

# Running LLAMA 2 on Colab - No UI

## Installing of required packages

Downloading quantized model from Google Drive

In [1]:
!pip install langchain==0.0.310 --quiet
!export LLAMA_CUBLAS=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python==0.2.11 --quiet
!pip install sentence-transformers==2.2.2 --quiet
!pip install chromadb==0.3.26 --quiet
!pip install ctransformers --quiet
!pip install pypdf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5 kB 8.5 MB/s eta 0:00:00
     ━

In [2]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf

--2024-01-18 01:36:46--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/e0b99920cf47b94c78d2fb06a1eceb9ed795176dfa3f7feac64629f1b52b997f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q5_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q5_K_M.gguf%22%3B&Expires=1705801006&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTgwMTAwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlL2UwYjk5OTIwY2Y0N2I5NGM3OGQyZmIwNmExZWNlYjllZDc5NTE

## Loading the model

In [3]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    n_ctx = 4096,
    model_path="/content/llama-2-7b-chat.Q5_K_M.gguf", # location of the model, llama-2-13b-chat.Q4_0.gguf
    temperature=0.2,                 # temperature
    max_tokens=2000,                 # Max. number of tokens to be generated
    top_p=0.9,                    # top_p = 0.9
    top_k=30,                     # top_k = 30
    n_gpu_layers=200,                 # number of layers to offload to GPU
    verbose=True, # Verbose is required to pass to the callback manager
    callback_manager=callback_manager,
    n_batch=200,          # number of token generation in parallel
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from chromadb.config import Settings

import pandas as pd
import os

In [5]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant."

def create_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
  SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
  prompt = B_INST + SYSTEM_PROMPT + instruction + E_INST
  return "" + prompt

In [6]:
qa_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful concise answer below and nothing else.
Helpful answer:"""

In [7]:
prompt_template = create_prompt(qa_template)
print(prompt_template)
prompt = PromptTemplate(template=prompt_template,input_variables=['context', 'question'])

[INST]<>
You are a helpful, respectful and honest assistant.
<>

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful concise answer below and nothing else.
Helpful answer:[/INST]


## Loading DocumentQA using Langchain

In [8]:
persist_directory = "./"

CHROMA_SETTINGS = Settings(
    chroma_db_impl = 'duckdb+parquet',  # use duckdb to r/w parquet file
    anonymized_telemetry = False     # refuse telemetry of usage info
)

model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}      # you must have a gpu, otherwise change it to cpu
encode_kwargs = {'normalize_embeddings': True}
embedding_func = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:
def setup_new_qa(filename):
  global llm
  # Loading of documents
  loader = PyPDFLoader(filename)
  documents = loader.load()
  # Splitting of text into chunks

  text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
  texts = text_splitter.split_documents(documents)

  # Embeddings to use, to transform the document
  # embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',
  #                                   model_kwargs={'device':'cpu'})

  text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
  chunked_docs = text_splitter.split_documents(documents)

  # configuration of the vector store

  print("Creating new vector store...")
  db = Chroma.from_documents(
      chunked_docs,
      embedding_func,
      persist_directory=persist_directory,
      client_settings=CHROMA_SETTINGS
  )

  db.persist()

  retriever = db.as_retriever(search_type = 'similarity', search_kwargs = {'k':4}) # with top k ranked matches

  qa = ConversationalRetrievalChain.from_llm(llm, retriever,verbose = True, return_source_documents=True, combine_docs_chain_kwargs={"prompt": prompt})

  return qa

## Gradio Installation

In [10]:
!pip install gradio==3.48.0 pydantic --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.


# UI for Document Q&A

In [11]:
import gradio as gr
import random
import time

COUNT = 0
chat_history = []
qa = None

def process_file(files):
    global qa
    file_paths = [file.name for file in files]
    qa = setup_new_qa(file_paths[0]) # btn.name


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    file_output = gr.File()
    btn = gr.UploadButton("Click to Upload a File", file_types=["PDF"], file_count="multiple")
    btn.upload(process_file, btn, file_output)

    used_letters_var = gr.State([])

    def respond(message, chat_history, btn):
        global COUNT, query, qa

        #'''
        #if not btn:
        #    raise gr.Error(message = 'Document Not Found.')

        #if COUNT == 0:
        #  #filename = "./datafile/ccop/CCOP.pdf"
        #  filename = btn.name
        #  qa = setup_new_qa(filename) # btn.name
        #  COUNT += 1


        req_msg = str(message)
        res_msg = ''
        try:
          if qa == None:
            return "", chat_history
          result = qa({'question': req_msg, 'chat_history': []})
          if result['answer'] == '':
            res_msg = 'No answer'
            return
          else:
            res_msg =  str(result['answer'][:1000])
        except:
          raise gr.Error('LLM hang!')

        bot_message = res_msg

        #print(message)
        #bot_message = message
        #chat_history = []
        chat_history.append((message, bot_message))
        #time.sleep(2)
        return "", chat_history

#     btn.upload(inputs = [btn])
    msg.submit(respond, [msg, chatbot, btn], [msg, chatbot])

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://36d249b907fc9e9006.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Creating new vector store...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST]<>
You are a helpful, respectful and honest assistant.
<>

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: Page 10 of 64 
 “scenario -based 
cybersecurity exercise”  An activity to assess and validate an organisation’s plans and 
capabilities relating to the handling of simulated 
cybersecurity incidents (the ‘scenario’) affecting the 
organisation. The exercise scenario should be based on 
relevant threats. Depending on the nature of the threats, CIIO 
may include social engineering and cyber range components 
in these scenarios.  
“security architecture”  A set of physical and logical cybersecurity representations 
that addresses potential cybersecurity risks and gaps in the 
environment.  
“secured  intermediary 
mechanism

Llama.generate: prefix-match hit


  I apologize, but I cannot confirm whether the Cybersecurity Code of Practice for Critical Information Infrastructure in Singapore is applicable to Australia as I'm just an AI assistant and do not have access to current Australian laws or regulations. The code you provided appears to be specific to Singapore and its critical information infrastructure, so it may not be directly applicable to Australia. It is essential to consult with relevant authorities or legal professionals for accurate advice on cybersecurity practices and compliance with Australian laws.
> Finished chain.

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://36d249b907fc9e9006.gradio.live


# Using AzureOpenAI as LLM

In [ ]:
!pip install openai --quiet
!pip install tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
# Setting up API access
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://dto-testing.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "ce8e5b0d756f4d83b4dfbbc4ccd08fec"


llm_OAI = AzureOpenAI( temperature = 0.1,
                   deployment_name="DTO_demo",  # This is the deployed GPT3 from Azure
                   model_name="text-davinci-003",
                            )

In [ ]:
qa_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful concise answer below and nothing else.
Helpful answer:"""

prompt = PromptTemplate(template=qa_template,input_variables=['context', 'question'])

In [ ]:
def setup_new_qa_OAI(filename):

  global llm_OAI
  # Loading of documents
  loader = PyPDFLoader(filename)
  documents = loader.load()
  # Splitting of text into chunks

  # Splitting of text into chunks

  text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
  texts = text_splitter.split_documents(documents)

  embeddings = OpenAIEmbeddings(deployment="DTO_embed"
                                ,model='text-embedding-ada-002'
                                ,chunk_size=1) # chunk_size number is peculiarity of Azure OpenAI

  db = Chroma.from_documents(texts, embeddings) # vector database

  retriever = db.as_retriever(search_type = 'similarity', search_kwargs = {'k':4}) # with top k ranked matches

  qa = ConversationalRetrievalChain.from_llm(llm_OAI, retriever,verbose = False, return_source_documents=True, combine_docs_chain_kwargs={"prompt": prompt})

  return qa

In [ ]:
import gradio as gr
import random
import time

COUNT = 0
chat_history = []
qa = None

def process_file(files):
    global qa
    file_paths = [file.name for file in files]
    qa = setup_new_qa(file_paths[0]) # btn.name


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    file_output = gr.File()
    btn = gr.UploadButton("Click to Upload a File", file_types=["PDF"], file_count="multiple")
    btn.upload(process_file, btn, file_output)

    used_letters_var = gr.State([])

    def respond(message, chat_history, btn):
        global COUNT, query, qa

        req_msg = str(message)
        res_msg = ''
        try:
          if qa == None:
            return "", chat_history
          result = qa({'question': req_msg, 'chat_history': []})
          if result['answer'] == '':
            res_msg = 'No answer'
            return
          else:
            res_msg =  str(result['answer'][:1000])
        except:
          raise gr.Error('LLM hang!')

        bot_message = res_msg
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot, btn], [msg, chatbot])

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://48f6a3392a2741170b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Creating new vector store...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: Page 5 of 64 
 In exercise of the powers conferred by section 11(1)(a) of the Cybersecurity Act (the “Act”), 
the Commissioner of Cybersecurity hereby issues the following Code:  
1 PRELIMINARY  
This section sets out the purpose and effect of this Code, compliance requirements  and 
timelines, and definitions of key terms .  
1.1 Citation and Commencement  
1.1.1 This Code  is the Cybersecurity Code of Practice for Critical Information Infrastructure  
(Second Edition ) and shall take effect from  4 July  2022, superseding previous versions of the 
Code.   
1.2 Glossary and Interpretation   
1.2.1 In this Code, unless the context  otherwise requires , the following terms shall have t

Llama.generate: prefix-match hit


 This document is about Cybersecurity Code of Practice for Critical Information Infrastructure.
> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: Page 47 of 64 
 7 RESPONSE AND RECOVERY REQUIREMENTS   
Establishing , managing , and exercising cybersecurity incident response plan s and crisis 
communication plans  to prepare the CII for cybersecurity incidents.  
7.1 Inciden t Management   
Incident management seeks  to minimise the impact of cybersecurity incidents through 
processes that include the identification, containment and eradication of cybersecurity threats, 
and the recovery of systems, root -cause analysis and implementation of corrective actions to 
prevent recurrence. It is important to have a plan with a clear

Llama.generate: prefix-match hit


...

 Page 34 of 64.
> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: Page 25 of 64 
 4.1.2 The CIIO shall update the inventory whenever there is any change to any CII asset or 
to the information to be recorded in the inventory .

Page 25 of 64 
 4.1.2 The CIIO shall update the inventory whenever there is any change to any CII asset or 
to the information to be recorded in the inventory .

Page 25 of 64 
 4.1.2 The CIIO shall update the inventory whenever there is any change to any CII asset or 
to the information to be recorded in the inventory .

Page 24 of 64 
 4 IDENTIFICATION REQUIREMENTS  
Identification requirements aim to assist the CIIO in understanding and identifying the 
resources and assets supporting the CIIO’s cri

Llama.generate: prefix-match hit


 I don't know, I'm just an AI and do not have personal preferences or taste buds to suggest food options for you.
> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: Page 62 of 64 
 ANNEX  A – GUIDANCE FOR STRENGTHENING ORGANISATIONAL 
CYBERSECURITY POSTURE  
An organisation’s cybersecurity is only as strong as its weakest link. Even as measures are 
taken under this code to ensure the cybersecurity of the CII, it is vital that CIIOs ensure 
cybersecurity throughout its organisation. This Annex A seeks to provide guidance for 
strengthening the cybersecurity posture of the organisation as a whole. CIIOs are highly 
encouraged to implement these measures.  
 
For the avoidance of doubt, the matters in  this Annex need not be inc

Llama.generate: prefix-match hit



The page numbers are not included in the scope of cybersecurity audits conducted under section 15 of the Act.
> Finished chain.

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://48f6a3392a2741170b.gradio.live
